In [2]:
!pip install tensorflow.contrib

ERROR: Could not find a version that satisfies the requirement tensorflow.contrib (from versions: none)
ERROR: No matching distribution found for tensorflow.contrib


In [3]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# # Assuming 'df' is your DataFrame with numeric columns
# df = pd.DataFrame({
#     'A': [10],
#     'B': [5],
#     'C': [100]
# })

# # Extract the numeric columns you want to normalize
# columns_to_normalize = ['A', 'B', 'C']
# data_to_normalize = df[columns_to_normalize]

# # Use StandardScaler to perform Z-score normalization
# scaler = StandardScaler()

# # Check for zero standard deviation before normalization
# std_dev_nonzero = data_to_normalize.std() != 0
# normalized_data = pd.DataFrame(scaler.fit_transform(data_to_normalize.loc[:, std_dev_nonzero]), columns=columns_to_normalize[std_dev_nonzero])

# # Replace the original columns with the normalized values
# df[columns_to_normalize[std_dev_nonzero]] = normalized_data

# # Print the original and normalized DataFrame
# print("Original DataFrame:")
# print(df)
# print("\nNormalized DataFrame (Z-score):")
# print(normalized_data)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Read Csv Files**

In [5]:
# Load the CSV file
csv_path_threat = '/content/drive/MyDrive/threat_cleaned_audio.csv'
csv_path_non_threat = '/content/drive/MyDrive/Non_Threat_Audio.csv'


In [6]:
df_threat = pd.read_csv(csv_path_threat)
df_non_threat = pd.read_csv(csv_path_non_threat)

In [7]:
# add label
df_non_threat.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,label
0,tr_10_tr01010.wav,0.456072,0.005838,1042.672385,1384.334158,2190.630079,0.010599,-579.669312,120.963036,-33.088383,...,7.257706,4.710307,1.988918,13.701883,1.335759,13.912835,11.086916,3.744203,11.076061,/content/drive/MyDrive/VoiceShieldDataset/Proc...
1,tr_1048_tr11049.wav,0.547524,0.024877,994.832390,1435.610682,1934.226457,0.014654,-450.752747,108.422676,-29.897911,...,-7.804052,5.326353,-8.022465,8.961793,-0.283949,3.295633,3.028080,-2.268670,7.489334,/content/drive/MyDrive/VoiceShieldDataset/Proc...
2,tr_10000_tr097082.wav,0.520539,0.152251,668.442579,1116.236249,1300.973511,0.013611,-302.267792,127.838348,-13.609920,...,5.110784,4.058471,1.626830,4.522898,-2.011986,7.047409,5.164358,5.808061,7.171641,/content/drive/MyDrive/VoiceShieldDataset/Proc...
3,tr_10002_tr097084.wav,0.645273,0.141353,694.545389,1204.582351,1372.355727,0.007707,-347.164337,109.256714,-5.807846,...,0.014916,15.632376,-0.073168,9.660697,2.784854,10.505596,9.157940,5.360936,10.828979,/content/drive/MyDrive/VoiceShieldDataset/Proc...
4,tr_10001_tr097083.wav,0.578857,0.147588,701.233852,1193.312482,1430.052807,0.010718,-314.344086,121.566399,-13.635324,...,1.316138,11.982705,-2.986043,6.102071,0.221080,8.692894,9.341649,5.502691,6.445087,/content/drive/MyDrive/VoiceShieldDataset/Proc...


In [8]:
df_threat['label'] = "Threat"
df_non_threat['label'] = "NonThreat"
df_non_threat.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,label
0,tr_10_tr01010.wav,0.456072,0.005838,1042.672385,1384.334158,2190.630079,0.010599,-579.669312,120.963036,-33.088383,...,7.257706,4.710307,1.988918,13.701883,1.335759,13.912835,11.086916,3.744203,11.076061,NonThreat
1,tr_1048_tr11049.wav,0.547524,0.024877,994.832390,1435.610682,1934.226457,0.014654,-450.752747,108.422676,-29.897911,...,-7.804052,5.326353,-8.022465,8.961793,-0.283949,3.295633,3.028080,-2.268670,7.489334,NonThreat
2,tr_10000_tr097082.wav,0.520539,0.152251,668.442579,1116.236249,1300.973511,0.013611,-302.267792,127.838348,-13.609920,...,5.110784,4.058471,1.626830,4.522898,-2.011986,7.047409,5.164358,5.808061,7.171641,NonThreat
3,tr_10002_tr097084.wav,0.645273,0.141353,694.545389,1204.582351,1372.355727,0.007707,-347.164337,109.256714,-5.807846,...,0.014916,15.632376,-0.073168,9.660697,2.784854,10.505596,9.157940,5.360936,10.828979,NonThreat
4,tr_10001_tr097083.wav,0.578857,0.147588,701.233852,1193.312482,1430.052807,0.010718,-314.344086,121.566399,-13.635324,...,1.316138,11.982705,-2.986043,6.102071,0.221080,8.692894,9.341649,5.502691,6.445087,NonThreat


In [9]:
print(len(df_threat))
print(len(df_non_threat))

2827
2972


In [10]:
# concat
import pandas as pd

# Concatenate along rows (vertically)
df = pd.concat([df_threat, df_non_threat], ignore_index=True)
df.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,label
0,0284.wav,0.386675,0.029384,1225.019293,1665.140182,2205.985857,0.050022,-404.056793,123.786240,8.928247,...,-1.359868,-0.215260,-11.296312,-12.897702,-15.708597,-14.613194,-15.897648,-6.177386,-7.488589,Threat
1,0285.wav,0.386317,0.026023,1367.611014,1719.643953,2318.374346,0.062399,-418.657501,112.347954,19.050705,...,1.226134,1.923700,-11.271539,-14.089395,-18.870596,-15.725780,-15.260133,-5.769762,-7.335375,Threat
2,0286.wav,0.378874,0.027696,1439.566809,1785.825066,2686.072513,0.068701,-402.853271,105.891815,14.659204,...,-5.765984,4.436638,-0.467150,-15.878686,-16.154736,-9.842570,-12.959028,-8.686703,-6.713859,Threat
3,0287.wav,0.372240,0.022424,1290.864194,1803.661083,2542.993458,0.045887,-426.939819,111.995491,22.286507,...,0.694303,6.483352,-3.414390,-13.434025,-15.002262,-11.482940,-13.211467,-6.688077,-5.732843,Threat
4,0288.wav,0.429877,0.031818,1586.090943,1865.992191,3339.332820,0.065489,-370.915741,99.107254,7.306797,...,-2.486910,-6.998872,-6.310315,-14.977116,-19.062798,-15.870350,-19.145720,-6.219707,-2.282526,Threat


In [11]:
df.to_csv('custome_data.csv', index=False, sep='\t', encoding='utf-8')

#**Audio Preprocessing**

In [12]:
# Drop multiple columns (e.g., 'B' and 'D')
# df.head()
columns_to_drop = ['filename', 'label']
labels = df.loc[:, 'label']
df_final = df.drop(columns=columns_to_drop)
df_final

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,0.386675,0.029384,1225.019293,1665.140182,2205.985857,0.050022,-404.056793,123.786240,8.928247,19.683321,-1.359868,-0.215260,-11.296312,-12.897702,-15.708597,-14.613194,-15.897648,-6.177386,-7.488589
1,0.386317,0.026023,1367.611014,1719.643953,2318.374346,0.062399,-418.657501,112.347954,19.050705,19.246691,1.226134,1.923700,-11.271539,-14.089395,-18.870596,-15.725780,-15.260133,-5.769762,-7.335375
2,0.378874,0.027696,1439.566809,1785.825066,2686.072513,0.068701,-402.853271,105.891815,14.659204,18.686604,-5.765984,4.436638,-0.467150,-15.878686,-16.154736,-9.842570,-12.959028,-8.686703,-6.713859
3,0.372240,0.022424,1290.864194,1803.661083,2542.993458,0.045887,-426.939819,111.995491,22.286507,21.629602,0.694303,6.483352,-3.414390,-13.434025,-15.002262,-11.482940,-13.211467,-6.688077,-5.732843
4,0.429877,0.031818,1586.090943,1865.992191,3339.332820,0.065489,-370.915741,99.107254,7.306797,26.858704,-2.486910,-6.998872,-6.310315,-14.977116,-19.062798,-15.870350,-19.145720,-6.219707,-2.282526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,0.234945,0.002263,770.460278,680.118838,1428.881836,0.056481,-702.099731,71.174545,-26.294155,17.791243,-14.648263,20.170454,-6.076617,-6.195958,8.396724,-5.033086,3.417111,-4.781724,2.236403
5795,0.141242,0.008330,613.503097,614.634883,1221.817017,0.033191,-618.715515,66.292068,-7.348810,8.735965,-8.491941,18.333899,-0.765503,-6.088050,5.850920,-4.849899,-2.004101,-2.683825,-4.208126
5796,0.217010,0.031295,925.518416,911.749967,1892.486572,0.051290,-449.214752,60.932327,-2.600382,16.911623,5.212480,8.998152,-4.573737,-1.067787,3.909385,-4.562554,-0.311636,-8.381059,-5.630166
5797,0.200627,0.025993,1085.145878,983.064078,2225.289917,0.053961,-466.797333,54.115669,-7.249406,25.861017,-13.273665,13.405861,-4.534163,1.607583,-0.488903,-3.546915,0.585653,-11.414605,-2.719762


In [13]:
# Use StandardScaler to perform Z-score normalization
scaler = StandardScaler()
df_normalized_zscore = pd.DataFrame(scaler.fit_transform(df_final), columns=df_final.columns)

# Save the trained scaler for later use
joblib.dump(scaler, 'scaler_model.pkl')

df_normalized_zscore

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,0.102101,-0.756937,-0.294703,0.231400,-0.439508,-0.376935,-0.212143,0.999253,0.763943,-0.342692,0.131892,-0.745449,-0.156639,-0.963019,-1.530693,-1.504573,-1.699037,-0.349483,-0.825480
1,0.099432,-0.834578,-0.070812,0.337072,-0.350990,-0.067093,-0.390114,0.552376,1.367830,-0.367676,0.337220,-0.558109,-0.154413,-1.077497,-1.932377,-1.638437,-1.616226,-0.282051,-0.804523
2,0.043811,-0.795938,0.042169,0.465383,-0.061387,0.090676,-0.197473,0.300145,1.105841,-0.399725,-0.217952,-0.338014,0.816247,-1.249382,-1.587369,-0.930582,-1.317322,-0.764591,-0.719512
3,-0.005764,-0.917734,-0.191316,0.499964,-0.174078,-0.480473,-0.491068,0.538606,1.560873,-0.231323,0.294993,-0.158753,0.551469,-1.014540,-1.440964,-1.127948,-1.350112,-0.433965,-0.585330
4,0.424931,-0.700695,0.272234,0.620811,0.453127,0.010262,0.191819,0.035083,0.667210,0.067894,0.042405,-1.339591,0.291300,-1.162774,-1.956793,-1.655831,-2.120950,-0.356484,-0.113397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,-1.031692,-1.383587,-1.008430,-1.678355,-1.051563,-0.215248,-3.845039,-1.056206,-1.337363,-0.450959,-0.923202,1.040030,0.312296,-0.319228,1.531518,-0.351916,0.809878,-0.118604,0.504700
5795,-1.731883,-1.243392,-1.254877,-1.805315,-1.214649,-0.798298,-2.828655,-1.246957,-0.207118,-0.969115,-0.434392,0.879176,0.789443,-0.308862,1.208112,-0.329875,0.105683,0.228444,-0.376779
5796,-1.165712,-0.712772,-0.764965,-1.229270,-0.686423,-0.345203,-0.762581,-1.456353,0.076165,-0.501292,0.653735,0.061506,0.447314,0.173401,0.961470,-0.295302,0.325528,-0.714029,-0.571285
5797,-1.288133,-0.835280,-0.514326,-1.091006,-0.424304,-0.278334,-0.976898,-1.722670,-0.201187,0.010805,-0.814060,0.447554,0.450869,0.430405,0.402735,-0.173103,0.442082,-1.215858,-0.173202


In [ ]:
# Z-score normalization


# df_normalized_zscore = (df_final - df_final.mean()) / df_final.std()
# df_normalized_zscore.head()

In [14]:
# Handling NaN values: Interpolation
df_interpolated = df_normalized_zscore.interpolate()
df_interpolated

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,0.102101,-0.756937,-0.294703,0.231400,-0.439508,-0.376935,-0.212143,0.999253,0.763943,-0.342692,0.131892,-0.745449,-0.156639,-0.963019,-1.530693,-1.504573,-1.699037,-0.349483,-0.825480
1,0.099432,-0.834578,-0.070812,0.337072,-0.350990,-0.067093,-0.390114,0.552376,1.367830,-0.367676,0.337220,-0.558109,-0.154413,-1.077497,-1.932377,-1.638437,-1.616226,-0.282051,-0.804523
2,0.043811,-0.795938,0.042169,0.465383,-0.061387,0.090676,-0.197473,0.300145,1.105841,-0.399725,-0.217952,-0.338014,0.816247,-1.249382,-1.587369,-0.930582,-1.317322,-0.764591,-0.719512
3,-0.005764,-0.917734,-0.191316,0.499964,-0.174078,-0.480473,-0.491068,0.538606,1.560873,-0.231323,0.294993,-0.158753,0.551469,-1.014540,-1.440964,-1.127948,-1.350112,-0.433965,-0.585330
4,0.424931,-0.700695,0.272234,0.620811,0.453127,0.010262,0.191819,0.035083,0.667210,0.067894,0.042405,-1.339591,0.291300,-1.162774,-1.956793,-1.655831,-2.120950,-0.356484,-0.113397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,-1.031692,-1.383587,-1.008430,-1.678355,-1.051563,-0.215248,-3.845039,-1.056206,-1.337363,-0.450959,-0.923202,1.040030,0.312296,-0.319228,1.531518,-0.351916,0.809878,-0.118604,0.504700
5795,-1.731883,-1.243392,-1.254877,-1.805315,-1.214649,-0.798298,-2.828655,-1.246957,-0.207118,-0.969115,-0.434392,0.879176,0.789443,-0.308862,1.208112,-0.329875,0.105683,0.228444,-0.376779
5796,-1.165712,-0.712772,-0.764965,-1.229270,-0.686423,-0.345203,-0.762581,-1.456353,0.076165,-0.501292,0.653735,0.061506,0.447314,0.173401,0.961470,-0.295302,0.325528,-0.714029,-0.571285
5797,-1.288133,-0.835280,-0.514326,-1.091006,-0.424304,-0.278334,-0.976898,-1.722670,-0.201187,0.010805,-0.814060,0.447554,0.450869,0.430405,0.402735,-0.173103,0.442082,-1.215858,-0.173202


In [15]:
df

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,label
0,0284.wav,0.386675,0.029384,1225.019293,1665.140182,2205.985857,0.050022,-404.056793,123.786240,8.928247,...,-1.359868,-0.215260,-11.296312,-12.897702,-15.708597,-14.613194,-15.897648,-6.177386,-7.488589,Threat
1,0285.wav,0.386317,0.026023,1367.611014,1719.643953,2318.374346,0.062399,-418.657501,112.347954,19.050705,...,1.226134,1.923700,-11.271539,-14.089395,-18.870596,-15.725780,-15.260133,-5.769762,-7.335375,Threat
2,0286.wav,0.378874,0.027696,1439.566809,1785.825066,2686.072513,0.068701,-402.853271,105.891815,14.659204,...,-5.765984,4.436638,-0.467150,-15.878686,-16.154736,-9.842570,-12.959028,-8.686703,-6.713859,Threat
3,0287.wav,0.372240,0.022424,1290.864194,1803.661083,2542.993458,0.045887,-426.939819,111.995491,22.286507,...,0.694303,6.483352,-3.414390,-13.434025,-15.002262,-11.482940,-13.211467,-6.688077,-5.732843,Threat
4,0288.wav,0.429877,0.031818,1586.090943,1865.992191,3339.332820,0.065489,-370.915741,99.107254,7.306797,...,-2.486910,-6.998872,-6.310315,-14.977116,-19.062798,-15.870350,-19.145720,-6.219707,-2.282526,Threat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,f2-07-01-02-39.wav,0.234945,0.002263,770.460278,680.118838,1428.881836,0.056481,-702.099731,71.174545,-26.294155,...,-14.648263,20.170454,-6.076617,-6.195958,8.396724,-5.033086,3.417111,-4.781724,2.236403,NonThreat
5795,f2-06-02-02-41.wav,0.141242,0.008330,613.503097,614.634883,1221.817017,0.033191,-618.715515,66.292068,-7.348810,...,-8.491941,18.333899,-0.765503,-6.088050,5.850920,-4.849899,-2.004101,-2.683825,-4.208126,NonThreat
5796,f2-06-01-02-57.wav,0.217010,0.031295,925.518416,911.749967,1892.486572,0.051290,-449.214752,60.932327,-2.600382,...,5.212480,8.998152,-4.573737,-1.067787,3.909385,-4.562554,-0.311636,-8.381059,-5.630166,NonThreat
5797,f2-06-02-02-54.wav,0.200627,0.025993,1085.145878,983.064078,2225.289917,0.053961,-466.797333,54.115669,-7.249406,...,-13.273665,13.405861,-4.534163,1.607583,-0.488903,-3.546915,0.585653,-11.414605,-2.719762,NonThreat


In [16]:
df_interpolated

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,0.102101,-0.756937,-0.294703,0.231400,-0.439508,-0.376935,-0.212143,0.999253,0.763943,-0.342692,0.131892,-0.745449,-0.156639,-0.963019,-1.530693,-1.504573,-1.699037,-0.349483,-0.825480
1,0.099432,-0.834578,-0.070812,0.337072,-0.350990,-0.067093,-0.390114,0.552376,1.367830,-0.367676,0.337220,-0.558109,-0.154413,-1.077497,-1.932377,-1.638437,-1.616226,-0.282051,-0.804523
2,0.043811,-0.795938,0.042169,0.465383,-0.061387,0.090676,-0.197473,0.300145,1.105841,-0.399725,-0.217952,-0.338014,0.816247,-1.249382,-1.587369,-0.930582,-1.317322,-0.764591,-0.719512
3,-0.005764,-0.917734,-0.191316,0.499964,-0.174078,-0.480473,-0.491068,0.538606,1.560873,-0.231323,0.294993,-0.158753,0.551469,-1.014540,-1.440964,-1.127948,-1.350112,-0.433965,-0.585330
4,0.424931,-0.700695,0.272234,0.620811,0.453127,0.010262,0.191819,0.035083,0.667210,0.067894,0.042405,-1.339591,0.291300,-1.162774,-1.956793,-1.655831,-2.120950,-0.356484,-0.113397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,-1.031692,-1.383587,-1.008430,-1.678355,-1.051563,-0.215248,-3.845039,-1.056206,-1.337363,-0.450959,-0.923202,1.040030,0.312296,-0.319228,1.531518,-0.351916,0.809878,-0.118604,0.504700
5795,-1.731883,-1.243392,-1.254877,-1.805315,-1.214649,-0.798298,-2.828655,-1.246957,-0.207118,-0.969115,-0.434392,0.879176,0.789443,-0.308862,1.208112,-0.329875,0.105683,0.228444,-0.376779
5796,-1.165712,-0.712772,-0.764965,-1.229270,-0.686423,-0.345203,-0.762581,-1.456353,0.076165,-0.501292,0.653735,0.061506,0.447314,0.173401,0.961470,-0.295302,0.325528,-0.714029,-0.571285
5797,-1.288133,-0.835280,-0.514326,-1.091006,-0.424304,-0.278334,-0.976898,-1.722670,-0.201187,0.010805,-0.814060,0.447554,0.450869,0.430405,0.402735,-0.173103,0.442082,-1.215858,-0.173202


In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
# from tensorflow.keras.callbacks import LearningRateScheduler

# # Load your data
# # df = pd.read_csv('your_dataset.csv')  # Replace with your actual data file
# # df.drop('filename', inplace=True, axis=1)
# labels = df['label']  # Assuming 'label' is the column name for the target variable
# X = df_interpolated.values  # Assuming df_interpolated contains only numerical data

# # Encode labels
# label_encoder = LabelEncoder()
# y = to_categorical(label_encoder.fit_transform(labels))

# # Split the data into training, validation, and test sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

# # Define the max sequence length
# max_sequence_length = 100  # Define the fixed length for sequences

# # Pad or truncate sequences for X_train
# X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')

# # Pad or truncate sequences for X_val
# X_val_padded = pad_sequences(X_val, maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')

# # Pad or truncate sequences for X_test
# X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')

# # Reshape the input data to include the timestep dimension
# X_train_padded = np.reshape(X_train_padded, (X_train_padded.shape[0], X_train_padded.shape[1], 1))
# X_val_padded = np.reshape(X_val_padded, (X_val_padded.shape[0], X_val_padded.shape[1], 1))
# X_test_padded = np.reshape(X_test_padded, (X_test_padded.shape[0], X_test_padded.shape[1], 1))

# # Ensure that the padded sequences have the correct shape
# print(X_train_padded.shape, X_val_padded.shape, X_test_padded.shape)

# # The shape should be (num_samples, max_sequence_length, num_features)
# num_features = X_train_padded.shape[2]  # Assuming the last dimension is the number of features

# # Define the learning rate schedule
# def lr_schedule(epoch):
#     lr = 1e-3
#     if epoch > 5:
#         lr *= 0.5
#     return lr

# # Define the LSTM model
# model = Sequential()
# model.add(LSTM(units=128, return_sequences=True, input_shape=(max_sequence_length, num_features)))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())
# model.add(LSTM(units=64))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())
# model.add(Dense(units=32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(units=y.shape[1], activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Define the learning rate scheduler
# lr_scheduler = LearningRateScheduler(lr_schedule)

# # Train the model
# history = model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_val_padded, y_val), callbacks=[lr_scheduler])

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
# print("Test Accuracy:", test_accuracy)


(4639, 100, 1) (580, 100, 1) (580, 100, 1)
Epoch 1/10
145/145 [==============================] - 38s 210ms/step - loss: 0.7799 - accuracy: 0.5033 - val_loss: 0.6930 - val_accuracy: 0.5224 - lr: 0.0010
Epoch 2/10
145/145 [==============================] - 30s 208ms/step - loss: 0.7371 - accuracy: 0.5089 - val_loss: 0.6941 - val_accuracy: 0.4776 - lr: 0.0010
Epoch 3/10
145/145 [==============================] - 29s 199ms/step - loss: 0.7224 - accuracy: 0.4975 - val_loss: 0.6922 - val_accuracy: 0.5224 - lr: 0.0010
Epoch 4/10
145/145 [==============================] - 34s 233ms/step - loss: 0.7040 - accuracy: 0.5193 - val_loss: 0.7149 - val_accuracy: 0.4776 - lr: 0.0010
Epoch 5/10
145/145 [==============================] - 30s 204ms/step - loss: 0.7048 - accuracy: 0.5124 - val_loss: 0.7173 - val_accuracy: 0.4776 - lr: 0.0010
Epoch 6/10
145/145 [==============================] - 29s 201ms/step - loss: 0.6992 - accuracy: 0.5143 - val_loss: 0.6933 - val_accuracy: 0.5224 - lr: 0.0010
Epoch 7/1

#**Neural Network Model**

In [17]:
from keras.regularizers import l2

# Convert labels to numeric values using LabelEncoder
X = df_interpolated
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(labels)
y = to_categorical(df['label_encoded'])

# print(X[1300:1323])
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

print(X_train.shape, X_val.shape, X_test.shape)

(4639, 19) (580, 19) (580, 19)


In [25]:
X

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,0.102101,-0.756937,-0.294703,0.231400,-0.439508,-0.376935,-0.212143,0.999253,0.763943,-0.342692,0.131892,-0.745449,-0.156639,-0.963019,-1.530693,-1.504573,-1.699037,-0.349483,-0.825480
1,0.099432,-0.834578,-0.070812,0.337072,-0.350990,-0.067093,-0.390114,0.552376,1.367830,-0.367676,0.337220,-0.558109,-0.154413,-1.077497,-1.932377,-1.638437,-1.616226,-0.282051,-0.804523
2,0.043811,-0.795938,0.042169,0.465383,-0.061387,0.090676,-0.197473,0.300145,1.105841,-0.399725,-0.217952,-0.338014,0.816247,-1.249382,-1.587369,-0.930582,-1.317322,-0.764591,-0.719512
3,-0.005764,-0.917734,-0.191316,0.499964,-0.174078,-0.480473,-0.491068,0.538606,1.560873,-0.231323,0.294993,-0.158753,0.551469,-1.014540,-1.440964,-1.127948,-1.350112,-0.433965,-0.585330
4,0.424931,-0.700695,0.272234,0.620811,0.453127,0.010262,0.191819,0.035083,0.667210,0.067894,0.042405,-1.339591,0.291300,-1.162774,-1.956793,-1.655831,-2.120950,-0.356484,-0.113397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,-1.031692,-1.383587,-1.008430,-1.678355,-1.051563,-0.215248,-3.845039,-1.056206,-1.337363,-0.450959,-0.923202,1.040030,0.312296,-0.319228,1.531518,-0.351916,0.809878,-0.118604,0.504700
5795,-1.731883,-1.243392,-1.254877,-1.805315,-1.214649,-0.798298,-2.828655,-1.246957,-0.207118,-0.969115,-0.434392,0.879176,0.789443,-0.308862,1.208112,-0.329875,0.105683,0.228444,-0.376779
5796,-1.165712,-0.712772,-0.764965,-1.229270,-0.686423,-0.345203,-0.762581,-1.456353,0.076165,-0.501292,0.653735,0.061506,0.447314,0.173401,0.961470,-0.295302,0.325528,-0.714029,-0.571285
5797,-1.288133,-0.835280,-0.514326,-1.091006,-0.424304,-0.278334,-0.976898,-1.722670,-0.201187,0.010805,-0.814060,0.447554,0.450869,0.430405,0.402735,-0.173103,0.442082,-1.215858,-0.173202


In [21]:
model_path = '/content/drive/MyDrive/model.tflite'

In [ ]:
import tensorflow as tf
import numpy as np

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare the input data
# Assume the input shape is (1, 224, 224, 3) and input type is float32
X_input = np.array(X)
input_shape = input_details[0]['shape']
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

# # Set the input tensor
# interpreter.set_tensor(input_details[0]['index'], input_data)

# # Run inference
# interpreter.invoke()

# # Get the output tensor
# output_data = interpreter.get_tensor(output_details[0]['index'])
# print(output_data)
print(X_input.shape)

In [18]:
model = Sequential()

model.add(Dense(X_train.shape[1],input_dim =X_train.shape[1],activation ='relu'))

model.add(Dense(512,activation ='relu'))

model.add(Dropout(0.2))

model.add(Dense(256,activation ='relu'))

model.add(Dropout(0.2))

model.add(Dense(128,activation ='relu'))

model.add(Dropout(0.2))

model.add(Dense(y_train.shape[1],activation ='softmax'))

model.compile(loss = 'binary_crossentropy',optimizer='adadelta',metrics=['accuracy'])

model.fit(X_train, y_train ,epochs = 32,batch_size = 16,verbose=1)

model.evaluate(X_test,y_test)

model.summary()

Epoch 1/32
290/290 [==============================] - 3s 6ms/step - loss: 0.7120 - accuracy: 0.4699
Epoch 2/32
290/290 [==============================] - 3s 12ms/step - loss: 0.7033 - accuracy: 0.4893
Epoch 3/32
290/290 [==============================] - 4s 14ms/step - loss: 0.6952 - accuracy: 0.5130
Epoch 4/32
290/290 [==============================] - 4s 13ms/step - loss: 0.6884 - accuracy: 0.5437
Epoch 5/32
290/290 [==============================] - 3s 10ms/step - loss: 0.6821 - accuracy: 0.5812
Epoch 6/32
290/290 [==============================] - 3s 11ms/step - loss: 0.6746 - accuracy: 0.6299
Epoch 7/32
290/290 [==============================] - 4s 15ms/step - loss: 0.6683 - accuracy: 0.6691
Epoch 8/32
290/290 [==============================] - 4s 13ms/step - loss: 0.6621 - accuracy: 0.7124
Epoch 9/32
290/290 [==============================] - 3s 10ms/step - loss: 0.6551 - accuracy: 0.7474
Epoch 10/32
290/290 [==============================] - 3s 11ms/step - loss: 0.6483 - accurac

In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # or tf.lite.OpsSet.SELECT_TF_OPS for TensorFlow operations
]
# Convert the model
tflite_model = converter.convert()

In [20]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import joblib

# Save the trained model
model_filename = 'audio_model_v2.pkl'
joblib.dump(model, model_filename)



['audio_model_v2.pkl']

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Make predictions
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
# print(y_test, y_pred_prob)

# Evaluate using various metrics
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred)
recall = recall_score(np.argmax(y_test, axis=1), y_pred)
f1 = f1_score(np.argmax(y_test, axis=1), y_pred)
roc_auc = roc_auc_score(y_test[:, 1], y_pred_prob[:, 1])  # Assuming binary classification

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")

19/19 [==============================] - 0s 3ms/step
Accuracy: 0.8482758620689655
Precision: 0.817629179331307
Recall: 0.9057239057239057
F1 Score: 0.8594249201277955
AUC-ROC: 0.9392392713947484


In [ ]:
model.evaluate(X_test, y_test)

19/19 [==============================] - 0s 3ms/step - loss: 0.5204 - accuracy: 0.8483


[0.5203715562820435, 0.8482758402824402]

In [ ]:
import joblib

# Load the trained model
model_filename = 'audio_model_v2.pkl'
loaded_model = joblib.load(model_filename)


custom_data_path = "con"


In [ ]:
class LoadModel:
    def load_cnn_model(self, cnn_model_path):
        cnn_model = joblib.load(cnn_model_path)
        return cnn_model
    def load_tfidf_vectorizer(self, tfidf_file_path):
        tfidf_vectorizer = joblib.load(tfidf_file_path)
        return tfidf_vectorizer
    def load_audio_model(self, model_path):
        return joblib.load(model_path)

    def extract_features(self, file_path):
        with open(file_path, "rb") as audio_file:
            audio, sample_rate = librosa.load(audio_file, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13), axis=1)
        return mfccs

In [ ]:
model_loader = LoadModel()

In [ ]:
cnn_model = model_loader.load_audio_model(model_filename)